# Case Study: Explaining House Values with Counterfactual Explanations

In this case study we explain the census tract (median) home values from the Boston metropolitan area with counterfactual explanations using alibi. We will closely follow the exposition in 
https://docs.seldon.io/projects/alibi/en/stable/examples/cfproto_housing.html
to which we refer for all details.

## 1. Getting started with Python and Jupyter Notebook

In this section, Jupyter Notebook and Python settings are initialized.

In [ ]:
# Notebook settings
###################

# resetting variables
get_ipython().magic('reset -sf') 

# formatting: cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# plotting
%matplotlib inline

## 2. Imports

We begin by initializing a seed for reproducability. Next we import all necessary libraries.

In [ ]:
import tensorflow as tf
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs

import numpy as np
import pandas as pd

# random seed
np.random.seed(42)
tf.random.set_seed(42)

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

import os
from sklearn.datasets import load_boston
from alibi.explainers import CounterFactualProto

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False

## 3. Import Data


In [ ]:
# import data
boston = load_boston()
data = boston.data

In [ ]:
# features and target variable
feature_names = boston.feature_names
target = boston.target
print(feature_names)
print(len(feature_names))

In [ ]:
# Categorical target variable; when higher than median, give =1 
y_cat = np.zeros((target.shape[0],))
y_cat[np.where(target > np.median(target))[0]] = 1

In [ ]:
# rapid check on y_cat
y_cat[:10]

In [ ]:
# median of the variable target 
np.median(target)

## 4. Data Explorative Analysis

In [ ]:
# data as dataframe - we join the target variables, as well
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df = pd.concat([df, pd.Series(target, name='target')], axis=1)

In [ ]:
# checking missing values
df.isnull().sum()

In [ ]:
# we describe data
pd.set_option('precision', 2)
df.describe()

In [ ]:
# Data visualizations

# histograms
df.hist(bins=10,figsize=(15, 15), grid=False)

# save the plot 
#plt.savefig('INSERT PATH')
plt.show()

## 5. Multivariate exploration

In [ ]:
# correlation analysis
pd.set_option('precision', 2)
df.drop(['CHAS'], axis=1).corr(method='pearson')

In [ ]:
# bivariate plots with high correlation features (>0.4)
features = df.drop(['CRIM', 'ZN', 'CHAS',  'AGE', 'DIS', 'RAD', 'B', 'target'], axis = 1)
plt.figure(figsize=(20, 5))

# i: index
for i, col in enumerate(features.columns):
    # 3 plots here hence 1, 3
    plt.subplot(1, 6, i+1)
    x = df[col]
    y = df.target
    plt.plot(x, y, 'o')
    
    # Create regression line
    plt.plot(np.unique(x), np.poly1d(np.polyfit(x, y, 1))(np.unique(x)))
    plt.title(col)
    plt.xlabel(col)
    plt.ylabel('target')

#plt.savefig('INSERT PATH')
plt.show()

## 6. Data Preprocessing 

In [ ]:
# Delecte categorical variable CHAS 
# CHAS = Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
data = np.delete(data, 3, 1)
feature_names = np.delete(feature_names, 3)
feature_names

In [ ]:
# Standardize data 
mu = data.mean(axis=0)
sigma = data.std(axis=0)
data = (data - mu) / sigma

## 7. Machine learning Modeling with Tensorflow

In [ ]:
# we split data into train and test, before categorizing y_train and y_test
idx = 475
x_train, y_train = data[:idx,:], y_cat[:idx]
x_test, y_test = data[idx:,:], y_cat[idx:]

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
# quick check on shapes
print(y_train.shape)
print(y_test.shape)

## 8. The Tensorflow Model

In [ ]:
# we compute two distinct metrics
from tensorflow import keras
METRICS = [ 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.AUC(name='auc')
]

In [ ]:
# neural network: model definition
def nn_model():
    x_in = Input(shape=(12,))
    x = Dense(40, activation='relu')(x_in)
    x = Dense(40, activation='relu')(x)
    x_out = Dense(2, activation='softmax')(x)
    nn = Model(inputs=x_in, outputs=x_out)
    nn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=METRICS)
    return nn

In [ ]:
# fitting and saving the model
nn = nn_model()
nn.summary()
nn.fit(x_train, y_train, batch_size=64, epochs=500, verbose=0)
nn.save('INSERT MODEL NAME.h5', save_format='h5')

In [ ]:
# loading the model and computing test performance
nn = load_model('INSERT MODEL NAME.h5')  
score = nn.evaluate(x_test, y_test, verbose=0)
print(nn.metrics_names)
print('Test accuracy: ', score[1])
print('Test AUC: ', score[2])

In [ ]:
# saving model weights
nn.save_weights('INSERT PATH')

## 9. Generation of Counterfactual Explanations

In [ ]:
# we choose a data point (whose outcome) to be explained
# in the notes we used x_test[1] and x_test[2]
X = x_test[1].reshape((1,) + x_test[1].shape)
shape = X.shape
X

In [ ]:
# unnormalizing X - original values
orig = X * sigma + mu
orig

In [ ]:
# checkpoint: use the fitted model or load another one
nn = load_model('INSERT MODEL NAME.h5')  

In [ ]:
# alibi imports
import alibi
from alibi.explainers import CounterFactual

In [ ]:
# setting the parameters for configurations 1,2
target_proba = 1.0
tol = 0.1
target_class = 'other'
max_iter = 1000
lam_init = 1
max_lam_steps = 100
learning_rate_init = 0.1 

In [ ]:
# preparation to compute counterfactuals
cf = CounterFactual(nn, 
                    target_proba=target_proba, 
                    target_class='other',
                    tol=tol, 
                    shape=shape,
                    #early_stop=50,
                    feature_range= (x_train.min(axis=0), x_train.max(axis=0)), 
                    max_iter=max_iter, 
                    lam_init=lam_init, 
                    max_lam_steps=max_lam_steps,
                    learning_rate_init=learning_rate_init)

In [ ]:
# computing counterfactuals
%%time
explanation = cf.explain(X)
counterfactual = explanation.cf['X'][0]
counterfactual

In [ ]:
# probability of X
explanation.orig_proba

In [ ]:
# probabilities of the counterfactual
explanation.cf['proba']

In [ ]:
# counterfactual data point
counterfactual

In [ ]:
# checking classes
print('Original prediction: {}'.format(explanation.orig_class))
print('Counterfactual prediction: {}'.format(explanation.cf['class']))

In [ ]:
# list of deltas with tolerance 1e-4 (see notes)
orig = X * sigma + mu
counterfactual = explanation.cf['X'] * sigma + mu
delta = counterfactual - orig
for i, f in enumerate(feature_names):
    if np.abs(delta[0][i]) > 1e-4:
        print('{}: {}'.format(f, delta[0][i]))

In [ ]:
# End of notebook